In [1]:
import pandas as pd

import geopandas as gpd

from shapely.geometry import Point, Polygon

import requests

In [2]:
data = pd.read_csv('data/DOB_Violations.csv')
print(f"Length with Nans: {len(data)}")

/Users/apple/.virtualenvs/python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (1,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Length with Nans: 2035211


# Removing missing or malformed Building Ids

In [3]:
data = data.dropna(subset=['BORO'])
print(f"Canaility without BORO Nans: {len(data)}")

data = data.dropna(subset=['BLOCK'])
print(f"Canaility without BLOCK Nans: {len(data)}")

data = data.dropna(subset=['LOT'])
print(f"Canaility without LOT Nans: {len(data)}")

Canaility without BORO Nans: 2035209
Canaility without BLOCK Nans: 2022372
Canaility without LOT Nans: 2022333


In [4]:
print(f"Canaility before cleaning: {len(data)}")
data.BORO = data.BORO.astype('str')
data = data[data.BORO.apply(lambda row: row.isnumeric())]
print(f"Canaility after cleaning: {len(data)}")

print(f"Canaility before cleaning: {len(data)}")
data.BLOCK = data.BLOCK.astype('str')
data = data[data.BLOCK.apply(lambda row: row.isnumeric())]
print(f"Canaility after cleaning: {len(data)}")

print(f"Canaility before cleaning: {len(data)}")
data.LOT = data.LOT.astype('str')
data = data[data.LOT.apply(lambda row: row.isnumeric())]
print(f"Canaility after cleaning: {len(data)}")

Canaility before cleaning: 2022333
Canaility after cleaning: 1988139
Canaility before cleaning: 1988139
Canaility after cleaning: 1500904
Canaility before cleaning: 1500904
Canaility after cleaning: 1499942


# Remove Malformed ISSUE_DATEs

In [5]:
data['ISSUE_DATE'] = data['ISSUE_DATE'].astype(str)

In [6]:
data = data[data.apply(lambda row: " " not in row['ISSUE_DATE'], axis=1)]

In [7]:
data = data.dropna(subset=['ISSUE_DATE'])

In [8]:
data['ISSUE_DATE'] = data['ISSUE_DATE'].astype(int)

ValueError: invalid literal for int() with base 10: 'nan'

In [9]:
data = data[data.apply(lambda row: int(row['ISSUE_DATE']) > 20000000, axis=1)]

ValueError: ("invalid literal for int() with base 10: 'nan'", 'occurred at index 1398206')

# Number of DOB violations

In [ ]:
data.groupby('VIOLATION_TYPE_CODE').count().sort_values('BORO')

# Format BBLs for violations dataset

In [ ]:
def format_bbl(parcel):
    borough = str(parcel['BORO'])
    block = str(parcel['BLOCK']).zfill(5)
    lot = str(parcel['LOT'])[-4:]
    
    return int(borough + block + lot)

data['BBL'] = data.apply(format_bbl, axis=1)

In [ ]:
data = data.set_index('BBL')
data.head()

In [ ]:
# Filter out elevator violations
elevator_data = data[data['VIOLATION_TYPE_CODE'] == 'E']

In [ ]:
print("Elevators are {}% of violations".format(len(elevator_data) / len(data) * 100))

In [ ]:
print("{} Elevator violations".format(len(elevator_data)))
elevator_data.head()

# Question: Do buildings have multiple violations?

In [ ]:
import matplotlib.pyplot as plot

num_violations = 20

grouped_elevator_violations = elevator_data.groupby('BBL').count().sort_values(by='BORO', ascending=False)
n_single_violations = len(grouped_elevator_violations[grouped_elevator_violations['BORO'] >= num_violations])
print("{}% of building with violation have atleast {} violations.".format(((n_single_violations / len(grouped_elevator_violations) * 100)), num_violations))

In [ ]:
plot.hist(grouped_elevator_violations['BORO'], bins=range(30))

Ok, so wheather or not you have a previous violation is probably a pretty good indicator...

# Question: Where are the elevator violations occuring?

In [ ]:
# Load up PLUTO for location info!
pluto = gpd.read_file('data/pluto.geojson')

In [ ]:
pluto = pluto.rename(columns={'bbl':'BBL'})
pluto = pluto.set_index('BBL')
pluto.head()

### Join PLUTO and elevator data to get location of elevators

In [ ]:
elevator_data_geo = elevator_data.join(pluto, how='inner', lsuffix='_left', rsuffix='_right')

In [ ]:
print(len(elevator_data))
print(len(elevator_data_geo))
# I guess this means some elevator bbls were not in pluto? We should validate that we formated the bbls correctly... using address?

### Create Long and lat columns for heat map

In [ ]:
elevator_data_geo['latitude'] = elevator_data_geo.apply(lambda row: row['geometry'].centroid.x, axis=1)
elevator_data_geo['longitude'] = elevator_data_geo.apply(lambda row: row['geometry'].centroid.y, axis=1)

In [ ]:
import folium
import geopandas
from folium import plugins

# convert to (n, 2) nd-array format for heatmap
violation_coords = elevator_data_geo[['longitude', 'latitude']].sample(frac=0.01).values
print(len(violation_coords))
# plot heatmap


m = folium.Map(location=[ 40.7128, -74.0060],zoom_start=11,tiles="Stamen Toner")
m.add_child(plugins.HeatMap(violation_coords, radius=15))
m

Ok, so I don't think this heatmap is that usefull but it looks cool.  I think a more useful measure would be (# elevator violations | boro / total # of violations 

In [ ]:
elevator_data.groupby('BORO').count().sort_values(by='LOT')

Had to see that coming.  Manhattan probs has the most elevators, therefore the most violations.

# Question: What is this DOB Inspection dataset

In [ ]:
from simpledbf import Dbf5

dbf = Dbf5('data/merge_footprints/merge_footprints.dbf')

df = dbf.to_dataframe()
print(len(df))

In [ ]:
elevator_inspections = df[df['Elevtrs'] != 0.0]
print("{} elevators inspections in the last 12 months".format(elevator_inspections['Elevtrs'].sum()))
print("{} buildings had elevator inspections in the last 12 months".format(len(elevator_inspections)))

### Mini question: what is the date range on the DOB Inspection dataset?

In [ ]:
print(df['dt_lstm'].min())
print(df['dt_lstm'].max())

df['dt_lstm'] = df['dt_lstm'].astype("datetime64")
df.groupby(df["dt_lstm"].dt.year).count()['bin'].plot(kind="bar")

### Mini question: what is the date range on the DOB Inspection dataset?

In [ ]:
data['ISSUE_DATE'] = data['ISSUE_DATE'].astype("datetime64")

print(data['ISSUE_DATE'].min())
print(data['ISSUE_DATE'].max())


# df.groupby(df["dt_lstm"].dt.year).count()['bin'].plot(kind="bar")

In [ ]:
for i in data['ISSUE_DATE']:
    print(i)
#     print(type(i))
    pd.to_datetime(i)
#     if " " in i:
#         print(i)
    
# pd.to_datetime()

# Question: What percentage of elevator inspections in the last 12 months resulted in a violation?

In [ ]:
list(elevator_inspections)

In [ ]:
# filter out elevator violations in the last 12 months

In [ ]:
elevator_data['ISSUE_DATE']

In [ ]:
elevator_inspections.apply(lambda row: row['bin'] in elevator_data['BIN'], axis=1)

In [ ]:
elevator_data['BIN']

# MAKE SURE WHAT EVER DATA YOU USE THE DATES LINE UP!
# Index by address instead of BBL?

# Box plots / stats sig tests of each feature for violation / no violations

# Question: When are the elevator violations occuring?